In [ ]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
from pathlib import Path
import hvplot.pandas
import seaborn as sns
%matplotlib inline

In [ ]:
#Set Tickers for stocks and crypto for analysis only looking at top 10 existing for at least 4 years
stock_tickers = ['MSFT','AAPL','TSLA','AMZN','NVDA','GOOG','FB','ADBE','NFLX','PYPL']
crypto_tickers = ['BTC-USD','ETH-USD','BNB-USD','ADA-USD','XRP-USD','DOGE-USD','LTC-USD','LINK-USD','BCH-USD','XLM-USD']
sp500 = ['^GSPC']

In [ ]:
#set tickers and parameters
tickers = stock_tickers + crypto_tickers + sp500
#1 day interal for data
interval= '1d'
#time to obtain data from y,m,d,hour,minute
period1 = int(time.mktime(datetime.datetime(2014, 11, 1, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime(2021, 11, 16, 23, 59).timetuple()))

In [ ]:
#loop through tickers to get close data from historical data
close_df = pd.DataFrame()
for ticker in tickers:
    query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=True'
    df = pd.read_csv(query_string)
    df = df.set_index('Date')
    close_df[f'{ticker}'] = df['Close']

In [ ]:
#drop na values
close_df = close_df.dropna()
close_df.head()

In [ ]:
#calculate daily return for all tickers
daily_df = close_df.pct_change().dropna()
daily_df

In [ ]:
daily_df[crypto_tickers].multiply(100).hvplot.line(title="Daily Returns", xlabel ='Date', ylabel='%Change')+daily_df[stock_tickers].multiply(100).hvplot.line(title="Daily Returns", xlabel ='Date', ylabel='%Change')+daily_df[sp500].multiply(100).hvplot.line(title="Daily Returns", xlabel ='Date', ylabel='%Change')

In [ ]:
#Calculate Standard Deviation tickers
std_tickers = daily_df.std()
std_tickers

In [ ]:
# Plot standard deviation for stocks and crypto
stocks_sd_plot = std_tickers[stock_tickers].hvplot.bar(
    xlabel='Stock/Crypto',
    rot=90,
    color='blue',    
    title='Standard Deviation for the Top 10 Tech Stocks and Cryptos')
crypto_sd_plot = std_tickers[crypto_tickers].hvplot.bar(
    rot=90,
    color='red')

stocks_sd_plot * crypto_sd_plot * std_tickers[sp500].hvplot.bar(
    rot=90,
    color='green')

In [ ]:
daily_df.hvplot.box(
    rot=90,
    title='Daily Returns Data for the Top 10 Tech Stocks and Cryptos')

In [ ]:
#calculate annual standard deviation for tickers
trading_days = 252
annual_std_tickers = std_tickers * trading_days ** (1/2)
annual_std_tickers

In [ ]:
# Calculate the annual average return data for tickers
annual_avg_return_tickers = daily_df.mean()* trading_days

annual_avg_return_tickers

In [ ]:
annual_avg_return_tickers[stock_tickers].hvplot.bar(title='Annual Average Returns', ylabel='Annual AVG Return', rot=90, color='blue')*annual_avg_return_tickers[crypto_tickers].hvplot.bar(title='Annual Average Returns', ylabel='Annual AVG Return', rot=90, color='red')*annual_avg_return_tickers[sp500].hvplot.bar(title='Annual Average Returns', ylabel='Annual AVG Return', rot=90, color='green')

In [ ]:
# Calculate the annualized Sharpe Ratios for tickers
annual_sharpe_ratio_tickers = annual_avg_return_tickers/annual_std_tickers

annual_sharpe_ratio_tickers

In [ ]:
# Plot annualized Sharpe Ratios for stocks and crypto
stocks_sharpe_plot = annual_sharpe_ratio_tickers[stock_tickers].hvplot.bar(
    xlabel='Stock/Crypto',
    rot=90,
    color='blue',    
    title='Sharpe Ratios for the Top 10 Tech Stocks and Cryptos')
crypto_sharpe_plot = annual_sharpe_ratio_tickers[crypto_tickers].hvplot.bar(
    rot=90,
    color='red')

stocks_sharpe_plot * crypto_sharpe_plot * annual_sharpe_ratio_tickers[sp500].hvplot.bar(
    rot=90,
    color='green')

In [ ]:
#for loop to calculate covariance (30day) for each crypto to each stock
covariance_crypto_stock_df = pd.DataFrame()

for stock in stock_tickers:
    for crypto in crypto_tickers:
        covariance_crypto_stock_df[f'{crypto} cov {stock}'] = daily_df[f'{crypto}'].rolling(window=30).cov(daily_df[f'{stock}'])
        
covariance_crypto_stock_df = covariance_crypto_stock_df.dropna()
covariance_crypto_stock_df.head()

In [ ]:
#for loop to calculate covariance (30day) for each ticker to market(^GSPC)
covariance_market_df = pd.DataFrame()

for ticker in tickers:
        covariance_market_df[f'{ticker}'] = daily_df[f'{ticker}'].rolling(window=30).cov(daily_df[f'{sp500[0]}'])
        
covariance_market_df = covariance_market_df.dropna()
covariance_market_df.head()

In [ ]:
#for loop to calculate covariance (30day) for each stock to each crypto
covariance_stock_crypto_df = pd.DataFrame()

for crypto in crypto_tickers:
    for stock in stock_tickers:
        covariance_stock_crypto_df[f'{stock} cov {crypto}'] = daily_df[f'{stock}'].rolling(window=30).cov(daily_df[f'{crypto}'])
        
covariance_stock_crypto_df = covariance_stock_crypto_df.dropna()
covariance_stock_crypto_df.head()

In [ ]:
#create function to group cov data by ticker stock or crypto.
def s_value(df, listmain, listrun):
    plots = {}
    for name in listmain:
        value = []
        for out in listrun:
            string = f"{name} cov {out}"
            value.append(string)
        plots[f"{name}"] = df[value].hvplot.scatter(title=f'{name} Covariance', xlabel='Date', ylabel='Covariance')
    return plots

In [ ]:
#call created function
cs_cov_plot = s_value(covariance_crypto_stock_df, crypto_tickers, stock_tickers)

In [ ]:
#view plots
cs_cov_plot['BTC-USD']+cs_cov_plot['ETH-USD']+cs_cov_plot['BNB-USD']+cs_cov_plot['ADA-USD']+cs_cov_plot['XRP-USD']+cs_cov_plot['LTC-USD']+cs_cov_plot['LINK-USD']+cs_cov_plot['BCH-USD']+cs_cov_plot['XLM-USD']+cs_cov_plot['DOGE-USD']

In [ ]:
#call created function
sc_cov_plot = s_value(covariance_stock_crypto_df, stock_tickers, crypto_tickers)

In [ ]:
sc_cov_plot['MSFT']+sc_cov_plot['AAPL']+sc_cov_plot['TSLA']+sc_cov_plot['AMZN']+sc_cov_plot['NVDA']+sc_cov_plot['GOOG']+sc_cov_plot['FB']+sc_cov_plot['ADBE']+sc_cov_plot['NFLX']+sc_cov_plot['PYPL']

In [ ]:
covariance_market_df[crypto_tickers].hvplot.scatter(title='Covariance to S&P 500 for Top 10 Cryptos')+covariance_market_df[stock_tickers].hvplot.scatter(title='Covariance to S&P 500 for Top 10 Stocks')

In [ ]:
#correlation between stocks and crypto sp500
price_correlation = close_df.corr()
price_correlation

In [ ]:
#heatmap to visulalize correlation
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(price_correlation, vmin=-1, vmax=1, cmap="mako", ax=ax)